In [ ]:
1

In [ ]:
import numpy as np
import pandas as pd

pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
import sys

sys.path.append("../technique")
from utils import *

In [ ]:
from scenario import (
    QfFiscalReform,
    StatutReform,
    CEReform,
    new_bracket,
    extract_max_value,
    SheetBasedReform,
)

In [ ]:
from centre_choregraphique import compute, build_data, get_df, fields, get_results

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

qf_fiscal_reform = QfFiscalReform(base)
statut_reform = StatutReform(qf_fiscal_reform)
ce_reform = CEReform(statut_reform)
reform = ce_reform

In [ ]:
recap, dfs = get_results(base, 10, reform)

In [ ]:
recap

In [ ]:
dfs[0][1]  # .groupby("sample_id").prix_r.sum().describe()

In [ ]:
recap.sort_values(("Recettes base", "Moyenne"), ascending=False)

In [ ]:
for v, res in dfs:
    print(v)
    display(res[~res.res])
    assert res[~res.res].shape[0] == 0

In [ ]:
recap[("Recettes base", "Moyenne")].sum()

In [ ]:
recap[("Recettes réforme", "Moyenne")].sum()

In [ ]:
recap[("Recettes réforme", "Moyenne")].sum() / recap[("Recettes base", "Moyenne")].sum()

In [ ]:
import seaborn
import matplotlib.pyplot as plt

In [ ]:
for i, v in enumerate([3, 4, 5]):
    print((i, v))

In [ ]:
qf_fiscal_reform = QfFiscalReform(base)

In [ ]:
(vv, _) = get_results(base, 2, qf_fiscal_reform)
vv.sort_values(("Recettes base", "Moyenne"), ascending=False)

In [ ]:
df = get_df()

In [ ]:
df[["Cours complet", "Nom"]].groupby(by="Cours complet").count().sort_values(
    "Nom", ascending=False
)

In [ ]:
df[["Cours complet", "Montant.facturé"]].groupby(by="Cours complet").describe()

In [ ]:
df[["Cours complet", "Montant.facturé"]].groupby(by="Cours complet").sum().sort_values(
    "Montant.facturé", ascending=False
)  # .cumsum()

In [ ]:
df[["Tarif", "Montant.facturé"]].groupby(by="Tarif").count()

In [ ]:
pd.pivot_table(df, columns="Tarif", values="Montant.facturé", aggfunc=len)

In [ ]:
pd.pivot_table(
    df, columns="Tarif", index="Cours complet", values="Montant.facturé", aggfunc=len
)

In [ ]:
df["qfrule"] = "QF_CCS_" + df.Tarif

In [ ]:
from openfisca_core.reforms import Reform

In [ ]:
base.parameters.communes.strasbourg.centre_choregraphique.enfant._1_cours

In [ ]:
import ezodf

In [ ]:
from scenario import new_bracket

In [ ]:
import ezodf

doc = ezodf.opendoc("../technique/scenario_ccs23.ods")

In [ ]:
import ezodf

doc = ezodf.opendoc("../technique/scenario_ccs23.ods")
res = []
for s in doc.sheets:
    sbr = SheetBasedReform(base, s)
    sbrr = StatutReform(sbr)
    v = get_results(base, 1, sbrr)
    res.append((s.name, v))

In [ ]:
import pandas as pd

In [ ]:
resumes = []
gdfs = []
for scenario, values in res:
    dfs = values[1]
    for service, df in dfs:
        df["service"] = service
    gdf = pd.concat([df for (_, df) in dfs], ignore_index=True)
    gdfs.append(gdf)
    resume = (
        gdf.groupby("sample_id")[["prix", "prix_r"]]
        .sum()
        .agg(["count", "average", "median", "std"])
        .rename(
            columns={"prix": "Recettes actuelles", "prix_r": "Recette simulées"},
            index={
                "count": "Nombre d'estimations",
                "average": "Moyenne des recettes",
                "median": "Médiane des recettes",
                "std": "Écart-type des recettes",
            },
        )
        .transpose()
        .reset_index(names="Indicateurs")
    )
    resume.insert(loc=0, column="Scénario", value=scenario)
    resumes.append(resume)

In [ ]:
columns = [
    "service",
    "sample_id",
    "individu_id",
    "qf_caf",
    "qf_fiscal",
    "prix_input",
    "prix",
    "prix_r",
]

In [ ]:
# file = pd.ExcelWriter('test4.ods')
file = pd.ExcelWriter("test4.xlsx")

In [ ]:
pd.concat(resumes, ignore_index=True).to_excel(file, sheet_name="Résumé")

In [ ]:
for i, (scenario, values) in enumerate(res):
    values[0].to_excel(file, sheet_name=scenario)
    gdfs[i][columns].to_excel(file, sheet_name=f"{scenario} détails")

In [ ]:
file.close()

In [ ]:
gdf[columns].to_excel("test2.ods", sheet_name="ok_details", index=False)

In [ ]:
class CCSReform(Reform):
    def apply(self):
        def modify_ce_parameters(local_parameters):
            P = local_parameters.communes.strasbourg.centre_choregraphique
            reductions = [
                (P.eveil, []),
                (
                    P.enfant._1_cours,
                    [
                        P.enfant._2_cours,
                        P.enfant._3_cours,
                        P.enfant._4_cours,
                    ],
                ),
                (
                    P.adulte._1_cours,
                    [
                        P.adulte._2_cours,
                        P.adulte._3_cours,
                        P.adulte._4_cours,
                    ],
                ),
            ]
            for node, nexts in reductions:
                tp = extract_max_value(node.TP)
                rb = tp * 0.5  # extract_max_value(node.RB)
                ra = tp * 0.8
                node.TP.brackets = [
                    new_bracket(threshold=0, amount=rb),
                    new_bracket(threshold=1, amount=ra),
                    new_bracket(threshold=1200, amount=tp),
                ]
                for i, n in enumerate(nexts):
                    coef = (i + 2) * (1 - (i + 2) * 0.05)
                    brackets = [
                        new_bracket(threshold=0, amount=rb * coef),
                        new_bracket(threshold=1, amount=tp * coef),
                    ]
                    n.TP.brackets = brackets
            return local_parameters

        self.modify_parameters(modifier_function=modify_ce_parameters)

In [ ]:
ccs_reform = CCSReform(qf_fiscal_reform)
statut_reform_r = StatutReform(ccs_reform)
ce_reform_r = CEReform(statut_reform_r)
full_reform = ce_reform_r

In [ ]:
recap_r, rdfs_r = get_results(base, 2, full_reform)
recap_r

In [ ]:
rdfs_r

In [ ]:
recap_r[("Recettes base", "Moyenne")].sum()

In [ ]:
recap_r[("Recettes réforme", "Moyenne")].sum()

In [ ]:
recap_r[("Recettes réforme", "Moyenne")].sum() / recap_r[
    ("Recettes base", "Moyenne")
].sum()

In [ ]:
recap_r[("Recettes réforme", "Moyenne")].sum() / recap_r[
    ("Recettes base", "Moyenne")
].sum()

In [ ]:
recap_r[("Recettes réforme", "Moyenne")].sum() / recap_r[
    ("Recettes base", "Moyenne")
].sum()

In [ ]:
recap_r[("Recettes réforme", "Moyenne")].sum() / recap_r[
    ("Recettes base", "Moyenne")
].sum()

In [ ]:
recap_r[("Recettes réforme", "Moyenne")].sum() / recap_r[
    ("Recettes base", "Moyenne")
].sum()